In [5]:
#Import necessary packages.

import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM, Embedding,Activation
from keras.models import load_model, Sequential
from keras.optimizers import Adam
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import json
import pandas as pd

%matplotlib inline

current_directory = os.getcwd()

In [6]:
#Load datasets.

X_train = np.load("Training_data.npy")
X_test = np.load("Test_data.npy")
Y_train = np.load('Training_labels.npy')
Y_test = np.load('Test_labels.npy')
human_train = np.load('Human_Test.npy')
human_test = np.load('Human_Test_Labels.npy')
human_data_reference = np.load('human_Data_Reference.npy')

human_test_dict = {}

for i in range(len(human_data_reference)):
    if human_data_reference[i][0] not in human_test_dict:
        human_test_dict[human_data_reference[i][0]] = [[], []]
    human_test_dict[human_data_reference[i][0]][0].append(human_train[i])
    human_test_dict[human_data_reference[i][0]][1].append(human_test[i])
for item in human_test_dict:
    human_test_dict[item][0] = np.array(human_test_dict[item][0])
    human_test_dict[item][1] = np.array(human_test_dict[item][1])
    
print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(len(Y_test))


(470250, 512)
(52250, 512)
470250
52250


In [7]:
#Regularize the data.

mean = X_train.mean(axis = 0)
X_train -= mean
X_test -= mean
for item in human_test_dict:
    human_test_dict[item][0] -= mean 

std = X_train.std(axis = 0)
X_train /= std
X_test /= std
for item in human_test_dict:
    human_test_dict[item][0] /= std

In [125]:
#[OPTIONAL] Choose a smaller sample.

train_data_selection = int(X_train.shape[0]*0.1)
test_data_selection = int(X_test.shape[0]*0.1)

X_train = X_train[:train_data_selection]
Y_train = Y_train[:train_data_selection]
X_test = X_test[:test_data_selection]
Y_test = Y_test[:test_data_selection]

print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(len(Y_test))

(47027, 512)
(5225, 512)
47027
5225


In [6]:
#Add one dimention to the input
X_train_test = np.array([[i] for i in X_train])

Y_train_test = np.array([[i] for i in Y_train])
X_train_test.shape

(470250, 1, 512)

In [8]:
#Train the LSTM network
model1 = Sequential()
model1.add(LSTM(128, return_sequences=True, input_shape= (1,X_train.shape[1])))
model1.add(Dropout(0.5))
model1.add(LSTM(128, return_sequences=False))
model1.add(Dropout(0.5))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])


history = model1.fit(X_train_test, Y_train_test, epochs = 1500, batch_size = 4096, shuffle = True,
                                validation_split = 0.1, verbose = 2)
   
model1.summary()
os.chdir('/data')
model1.save('LSTMmodel.h5')
        
os.chdir('/data')
with open('LSTM_history.pickle', 'wb') as dictionary:
    pickle.dump(history.history, dictionary, protocol=pickle.HIGHEST_PROTOCOL)


Train on 423225 samples, validate on 47025 samples
Epoch 1/1500
 - 5s - loss: 0.5998 - acc: 0.6753 - val_loss: 0.5820 - val_acc: 0.6881
Epoch 2/1500
 - 4s - loss: 0.5806 - acc: 0.6916 - val_loss: 0.5742 - val_acc: 0.6955
Epoch 3/1500
 - 4s - loss: 0.5732 - acc: 0.6972 - val_loss: 0.5713 - val_acc: 0.6959
Epoch 4/1500
 - 4s - loss: 0.5675 - acc: 0.7013 - val_loss: 0.5648 - val_acc: 0.7026
Epoch 5/1500
 - 4s - loss: 0.5626 - acc: 0.7051 - val_loss: 0.5612 - val_acc: 0.7048
Epoch 6/1500
 - 4s - loss: 0.5582 - acc: 0.7070 - val_loss: 0.5577 - val_acc: 0.7065
Epoch 7/1500
 - 4s - loss: 0.5536 - acc: 0.7109 - val_loss: 0.5551 - val_acc: 0.7090
Epoch 8/1500
 - 4s - loss: 0.5502 - acc: 0.7142 - val_loss: 0.5523 - val_acc: 0.7113
Epoch 9/1500
 - 4s - loss: 0.5465 - acc: 0.7162 - val_loss: 0.5514 - val_acc: 0.7117
Epoch 10/1500
 - 4s - loss: 0.5438 - acc: 0.7178 - val_loss: 0.5485 - val_acc: 0.7134
Epoch 11/1500
 - 4s - loss: 0.5406 - acc: 0.7198 - val_loss: 0.5456 - val_acc: 0.7155
Epoch 12/150

Epoch 96/1500
 - 4s - loss: 0.4496 - acc: 0.7772 - val_loss: 0.5216 - val_acc: 0.7400
Epoch 97/1500
 - 4s - loss: 0.4478 - acc: 0.7777 - val_loss: 0.5213 - val_acc: 0.7398
Epoch 98/1500
 - 4s - loss: 0.4471 - acc: 0.7790 - val_loss: 0.5200 - val_acc: 0.7398
Epoch 99/1500
 - 4s - loss: 0.4482 - acc: 0.7774 - val_loss: 0.5210 - val_acc: 0.7392
Epoch 100/1500
 - 4s - loss: 0.4469 - acc: 0.7788 - val_loss: 0.5226 - val_acc: 0.7382
Epoch 101/1500
 - 4s - loss: 0.4461 - acc: 0.7791 - val_loss: 0.5201 - val_acc: 0.7390
Epoch 102/1500
 - 4s - loss: 0.4457 - acc: 0.7796 - val_loss: 0.5218 - val_acc: 0.7407
Epoch 103/1500
 - 4s - loss: 0.4452 - acc: 0.7793 - val_loss: 0.5205 - val_acc: 0.7406
Epoch 104/1500
 - 4s - loss: 0.4444 - acc: 0.7800 - val_loss: 0.5215 - val_acc: 0.7392
Epoch 105/1500
 - 4s - loss: 0.4444 - acc: 0.7801 - val_loss: 0.5219 - val_acc: 0.7399
Epoch 106/1500
 - 4s - loss: 0.4443 - acc: 0.7808 - val_loss: 0.5216 - val_acc: 0.7402
Epoch 107/1500
 - 4s - loss: 0.4435 - acc: 0.78

Epoch 191/1500
 - 4s - loss: 0.4138 - acc: 0.7988 - val_loss: 0.5273 - val_acc: 0.7429
Epoch 192/1500
 - 4s - loss: 0.4141 - acc: 0.7988 - val_loss: 0.5273 - val_acc: 0.7429
Epoch 193/1500
 - 4s - loss: 0.4136 - acc: 0.7991 - val_loss: 0.5279 - val_acc: 0.7428
Epoch 194/1500
 - 4s - loss: 0.4130 - acc: 0.7992 - val_loss: 0.5274 - val_acc: 0.7418
Epoch 195/1500
 - 4s - loss: 0.4132 - acc: 0.7992 - val_loss: 0.5280 - val_acc: 0.7426
Epoch 196/1500
 - 4s - loss: 0.4130 - acc: 0.7998 - val_loss: 0.5277 - val_acc: 0.7434
Epoch 197/1500
 - 4s - loss: 0.4126 - acc: 0.8000 - val_loss: 0.5281 - val_acc: 0.7433
Epoch 198/1500
 - 4s - loss: 0.4118 - acc: 0.7995 - val_loss: 0.5295 - val_acc: 0.7438
Epoch 199/1500
 - 4s - loss: 0.4123 - acc: 0.7997 - val_loss: 0.5271 - val_acc: 0.7437
Epoch 200/1500
 - 4s - loss: 0.4110 - acc: 0.8010 - val_loss: 0.5282 - val_acc: 0.7436
Epoch 201/1500
 - 4s - loss: 0.4114 - acc: 0.7997 - val_loss: 0.5287 - val_acc: 0.7442
Epoch 202/1500
 - 4s - loss: 0.4123 - acc: 

 - 4s - loss: 0.3942 - acc: 0.8108 - val_loss: 0.5366 - val_acc: 0.7428
Epoch 286/1500
 - 4s - loss: 0.3934 - acc: 0.8112 - val_loss: 0.5359 - val_acc: 0.7444
Epoch 287/1500
 - 4s - loss: 0.3938 - acc: 0.8105 - val_loss: 0.5352 - val_acc: 0.7441
Epoch 288/1500
 - 4s - loss: 0.3931 - acc: 0.8111 - val_loss: 0.5364 - val_acc: 0.7428
Epoch 289/1500
 - 4s - loss: 0.3936 - acc: 0.8104 - val_loss: 0.5357 - val_acc: 0.7441
Epoch 290/1500
 - 4s - loss: 0.3932 - acc: 0.8112 - val_loss: 0.5364 - val_acc: 0.7433
Epoch 291/1500
 - 4s - loss: 0.3927 - acc: 0.8107 - val_loss: 0.5373 - val_acc: 0.7448
Epoch 292/1500
 - 4s - loss: 0.3927 - acc: 0.8118 - val_loss: 0.5348 - val_acc: 0.7434
Epoch 293/1500
 - 4s - loss: 0.3921 - acc: 0.8115 - val_loss: 0.5367 - val_acc: 0.7431
Epoch 294/1500
 - 4s - loss: 0.3927 - acc: 0.8105 - val_loss: 0.5361 - val_acc: 0.7433
Epoch 295/1500
 - 4s - loss: 0.3928 - acc: 0.8116 - val_loss: 0.5366 - val_acc: 0.7435
Epoch 296/1500
 - 4s - loss: 0.3923 - acc: 0.8110 - val_lo

Epoch 380/1500
 - 4s - loss: 0.3800 - acc: 0.8185 - val_loss: 0.5447 - val_acc: 0.7432
Epoch 381/1500
 - 4s - loss: 0.3796 - acc: 0.8187 - val_loss: 0.5420 - val_acc: 0.7426
Epoch 382/1500
 - 4s - loss: 0.3790 - acc: 0.8189 - val_loss: 0.5431 - val_acc: 0.7441
Epoch 383/1500
 - 4s - loss: 0.3785 - acc: 0.8199 - val_loss: 0.5431 - val_acc: 0.7437
Epoch 384/1500
 - 4s - loss: 0.3790 - acc: 0.8186 - val_loss: 0.5436 - val_acc: 0.7424
Epoch 385/1500
 - 4s - loss: 0.3793 - acc: 0.8184 - val_loss: 0.5426 - val_acc: 0.7428
Epoch 386/1500
 - 4s - loss: 0.3785 - acc: 0.8192 - val_loss: 0.5431 - val_acc: 0.7439
Epoch 387/1500
 - 4s - loss: 0.3783 - acc: 0.8190 - val_loss: 0.5433 - val_acc: 0.7442
Epoch 388/1500
 - 4s - loss: 0.3781 - acc: 0.8192 - val_loss: 0.5442 - val_acc: 0.7426
Epoch 389/1500
 - 4s - loss: 0.3781 - acc: 0.8197 - val_loss: 0.5442 - val_acc: 0.7443
Epoch 390/1500
 - 4s - loss: 0.3772 - acc: 0.8200 - val_loss: 0.5457 - val_acc: 0.7429
Epoch 391/1500
 - 4s - loss: 0.3766 - acc: 

...

Epoch 1041/1500
 - 4s - loss: 0.3310 - acc: 0.8462 - val_loss: 0.5892 - val_acc: 0.7409
Epoch 1042/1500
 - 4s - loss: 0.3309 - acc: 0.8460 - val_loss: 0.5908 - val_acc: 0.7408
Epoch 1043/1500
 - 4s - loss: 0.3318 - acc: 0.8461 - val_loss: 0.5876 - val_acc: 0.7409
Epoch 1044/1500
 - 4s - loss: 0.3310 - acc: 0.8461 - val_loss: 0.5912 - val_acc: 0.7403
Epoch 1045/1500
 - 4s - loss: 0.3313 - acc: 0.8457 - val_loss: 0.5903 - val_acc: 0.7400
Epoch 1046/1500
 - 4s - loss: 0.3313 - acc: 0.8461 - val_loss: 0.5909 - val_acc: 0.7400
Epoch 1047/1500
 - 4s - loss: 0.3313 - acc: 0.8458 - val_loss: 0.5896 - val_acc: 0.7408
Epoch 1048/1500
 - 4s - loss: 0.3299 - acc: 0.8467 - val_loss: 0.5916 - val_acc: 0.7402
Epoch 1049/1500
 - 4s - loss: 0.3310 - acc: 0.8463 - val_loss: 0.5922 - val_acc: 0.7404
Epoch 1050/1500
 - 4s - loss: 0.3299 - acc: 0.8469 - val_loss: 0.5935 - val_acc: 0.7412
Epoch 1051/1500
 - 4s - loss: 0.3301 - acc: 0.8467 - val_loss: 0.5907 - val_acc: 0.7398
Epoch 1052/1500
 - 4s - loss: 0.

 - 4s - loss: 0.3283 - acc: 0.8472 - val_loss: 0.5954 - val_acc: 0.7392
Epoch 1135/1500
 - 4s - loss: 0.3279 - acc: 0.8483 - val_loss: 0.5956 - val_acc: 0.7395
Epoch 1136/1500
 - 4s - loss: 0.3277 - acc: 0.8477 - val_loss: 0.5970 - val_acc: 0.7400
Epoch 1137/1500
 - 4s - loss: 0.3274 - acc: 0.8484 - val_loss: 0.5939 - val_acc: 0.7396
Epoch 1138/1500
 - 4s - loss: 0.3271 - acc: 0.8486 - val_loss: 0.5965 - val_acc: 0.7395
Epoch 1139/1500
 - 4s - loss: 0.3270 - acc: 0.8482 - val_loss: 0.5971 - val_acc: 0.7403
Epoch 1140/1500
 - 4s - loss: 0.3275 - acc: 0.8476 - val_loss: 0.5951 - val_acc: 0.7419
Epoch 1141/1500
 - 4s - loss: 0.3268 - acc: 0.8477 - val_loss: 0.5976 - val_acc: 0.7403
Epoch 1142/1500
 - 4s - loss: 0.3270 - acc: 0.8487 - val_loss: 0.5978 - val_acc: 0.7398
Epoch 1143/1500
 - 4s - loss: 0.3269 - acc: 0.8484 - val_loss: 0.5975 - val_acc: 0.7400
Epoch 1144/1500
 - 4s - loss: 0.3270 - acc: 0.8484 - val_loss: 0.5979 - val_acc: 0.7395
Epoch 1145/1500
 - 4s - loss: 0.3265 - acc: 0.84

Epoch 1228/1500
 - 4s - loss: 0.3232 - acc: 0.8507 - val_loss: 0.6017 - val_acc: 0.7402
Epoch 1229/1500
 - 4s - loss: 0.3244 - acc: 0.8500 - val_loss: 0.6019 - val_acc: 0.7385
Epoch 1230/1500
 - 4s - loss: 0.3245 - acc: 0.8499 - val_loss: 0.5985 - val_acc: 0.7398
Epoch 1231/1500
 - 4s - loss: 0.3248 - acc: 0.8493 - val_loss: 0.5983 - val_acc: 0.7385
Epoch 1232/1500
 - 4s - loss: 0.3229 - acc: 0.8508 - val_loss: 0.6057 - val_acc: 0.7385
Epoch 1233/1500
 - 4s - loss: 0.3237 - acc: 0.8504 - val_loss: 0.6003 - val_acc: 0.7378
Epoch 1234/1500
 - 4s - loss: 0.3231 - acc: 0.8502 - val_loss: 0.6016 - val_acc: 0.7390
Epoch 1235/1500
 - 4s - loss: 0.3229 - acc: 0.8505 - val_loss: 0.6041 - val_acc: 0.7385
Epoch 1236/1500
 - 4s - loss: 0.3237 - acc: 0.8499 - val_loss: 0.6023 - val_acc: 0.7376
Epoch 1237/1500
 - 4s - loss: 0.3246 - acc: 0.8493 - val_loss: 0.6033 - val_acc: 0.7382
Epoch 1238/1500
 - 4s - loss: 0.3246 - acc: 0.8495 - val_loss: 0.6019 - val_acc: 0.7384
Epoch 1239/1500
 - 4s - loss: 0.

 - 4s - loss: 0.3211 - acc: 0.8518 - val_loss: 0.6070 - val_acc: 0.7377
Epoch 1322/1500
 - 4s - loss: 0.3213 - acc: 0.8515 - val_loss: 0.6056 - val_acc: 0.7384
Epoch 1323/1500
 - 4s - loss: 0.3202 - acc: 0.8524 - val_loss: 0.6066 - val_acc: 0.7386
Epoch 1324/1500
 - 4s - loss: 0.3219 - acc: 0.8507 - val_loss: 0.6055 - val_acc: 0.7382
Epoch 1325/1500
 - 4s - loss: 0.3211 - acc: 0.8511 - val_loss: 0.6069 - val_acc: 0.7386
Epoch 1326/1500
 - 4s - loss: 0.3199 - acc: 0.8528 - val_loss: 0.6106 - val_acc: 0.7381
Epoch 1327/1500
 - 4s - loss: 0.3205 - acc: 0.8512 - val_loss: 0.6071 - val_acc: 0.7382
Epoch 1328/1500
 - 4s - loss: 0.3199 - acc: 0.8519 - val_loss: 0.6099 - val_acc: 0.7383
Epoch 1329/1500
 - 4s - loss: 0.3216 - acc: 0.8513 - val_loss: 0.6062 - val_acc: 0.7392
Epoch 1330/1500
 - 4s - loss: 0.3196 - acc: 0.8517 - val_loss: 0.6103 - val_acc: 0.7371
Epoch 1331/1500
 - 4s - loss: 0.3211 - acc: 0.8510 - val_loss: 0.6065 - val_acc: 0.7374
Epoch 1332/1500
 - 4s - loss: 0.3200 - acc: 0.85

Epoch 1415/1500
 - 4s - loss: 0.3166 - acc: 0.8540 - val_loss: 0.6111 - val_acc: 0.7381
Epoch 1416/1500
 - 4s - loss: 0.3180 - acc: 0.8537 - val_loss: 0.6131 - val_acc: 0.7380
Epoch 1417/1500
 - 4s - loss: 0.3178 - acc: 0.8533 - val_loss: 0.6129 - val_acc: 0.7376
Epoch 1418/1500
 - 4s - loss: 0.3179 - acc: 0.8536 - val_loss: 0.6116 - val_acc: 0.7375
Epoch 1419/1500
 - 4s - loss: 0.3174 - acc: 0.8535 - val_loss: 0.6144 - val_acc: 0.7383
Epoch 1420/1500
 - 4s - loss: 0.3183 - acc: 0.8536 - val_loss: 0.6109 - val_acc: 0.7382
Epoch 1421/1500
 - 4s - loss: 0.3174 - acc: 0.8537 - val_loss: 0.6112 - val_acc: 0.7383
Epoch 1422/1500
 - 4s - loss: 0.3178 - acc: 0.8533 - val_loss: 0.6125 - val_acc: 0.7371
Epoch 1423/1500
 - 4s - loss: 0.3182 - acc: 0.8528 - val_loss: 0.6114 - val_acc: 0.7384
Epoch 1424/1500
 - 4s - loss: 0.3181 - acc: 0.8528 - val_loss: 0.6127 - val_acc: 0.7384
Epoch 1425/1500
 - 4s - loss: 0.3171 - acc: 0.8536 - val_loss: 0.6135 - val_acc: 0.7375
Epoch 1426/1500
 - 4s - loss: 0.